In [1]:
import numpy as np
from numpy import sqrt, cos, sin,exp
import matplotlib
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from scipy.integrate import quad
from scipy.stats import vonmises
import math

In [2]:
def weibull(lams, beta, delta, gama):
    return (beta/delta)*((lams-gama)/delta)**(beta-1)*np.exp(-((lams-gama)/delta)**beta)

In [3]:
def integrand(lams, lam, beta, delta, gama):
    p_lams=(beta/delta)*((lams-gama)/delta)**(beta-1)*np.exp(-((lams-gama)/delta)**beta)
    lamr = lam/lams
    return p_lams*(lamr**2-(1/lamr))

In [4]:
def PK(F):
    C=np.dot(F,F.transpose())
    J=np.sqrt(np.linalg.det(C))
    I1= np.trace(C)
    cinv = np.linalg.inv(C)
    Sp=np.zeros([3,3])
    Id=np.eye(3)
    # W Isochoric
    Siso=2*(J**(-2/3))*(mu0)*(Id-(1/3)*I1*(cinv))

    # W Volume
    #Wvol = p*(J-1)
    p = 2*K*(J-1)
    Svol = p*J*cinv

    for i in range(10):
        theta=(i/10)*math.pi
        a0 = np.array([math.cos(theta),math.sin(theta),0])
        d_a0 = np.outer(a0,a0)
        lam = np.sqrt(np.dot(a0,np.dot(C,a0)))
       
        
        int_p = quad(integrand, gama, lam, args=(lam,beta,delta,gama))
        int_val = int_p[0]
        Sfiber = 2*mu1*(1/(lam**2))*int_val
        
        VM = vonmises.pdf(theta, kappa)
    
        Sp += Sfiber*d_a0*VM
        
    S = Siso + Svol + Sp
    return S

In [5]:
def CS(F,PK):
    C=np.dot(F,F.transpose())
    J=np.sqrt(np.linalg.det(C))
    CS = (1/J)*np.dot(F,np.dot(PK,(F.transpose())))
    return CS

In [6]:

mu0=0.41
mu1=225.93


beta=5.53
delta=0.44 
gama=0.97
K = 1
kappa=3

F = np.array([[1.2, 0, 0], [0, 1, 0], [0, 0, 1] ])

PK1 = PK(F)
print(PK1)
CS1 = CS(F,PK1)
print(CS1)

[[1.28790454 0.10152981 0.        ]
 [0.10152981 0.41496179 0.        ]
 [0.         0.         0.373498  ]]
[[1.54548545 0.10152981 0.        ]
 [0.10152981 0.34580149 0.        ]
 [0.         0.         0.31124833]]


# Elasticity Tensor

In [7]:
def integrand1(lams, lam, beta, delta, gama):
    p_lams=(beta/delta)*((lams-gama)/delta)**(beta-1)*np.exp(-((lams-gama)/delta)**beta)
    lamr = lam/lams
    return p_lams*(2*lamr**2+(1/lamr))

In [8]:
def integrand2(lams, lam, beta, delta, gama):
    p_lams=(beta/delta)*((lams-gama)/delta)**(beta-1)*np.exp(-((lams-gama)/delta)**beta)
    lamr = lam/lams
    return p_lams*(lamr**2-(1/lamr))

In [9]:
def ET(F):
    C=np.dot(F,F.transpose())
    J=np.sqrt(np.linalg.det(C))
    I1= np.trace(C)
    cinv = np.linalg.inv(C)
    Ciso=np.zeros([3,3,3,3])
    Cvol=np.zeros([3,3,3,3])
    Cplus=np.zeros([3,3,3,3])

    # Isochoric
    Siso = 2*(J**(-2/3))*(mu0/2)*(Id-I1*(cinv))
    for a in range(3):
        for b in range(3):
            for c in range(3):
                for d in range(3):
                    Ciso[a,b,c,d] = 2*(J**(-2./3))*mu0*((-2./3)*Id[a,b]*cinv[c,d] + (2./9)*I1*cinv[a,b]*cinv[c,d] - (2./3)*cinv[a,b]*Id[c,d] + (2./3)*cinv[a,c]*cinv[b,d])
    

    # Volume
    p = 2*K*(J-1)
    Svol = p*J*cinv
    for e in range(3):
        for f in range(3):
            for g in range(3):
                for h in range(3):
                    Cvol[e,f,g,h] =2.*((1/2)*p*J*cinv[e,f]*cinv[g,h] - p*J*cinv[e,g]*cinv[f,h])

    for i in range(10):
        theta=(i/10)*math.pi
        a0 = np.array([math.cos(theta),math.sin(theta),0])
        d_a0 = np.outer(a0,a0)
        lam = np.sqrt(np.dot(a0,np.dot(C,a0)))
       
        
        int_p1 = quad(integrand1, gama, lam, args=(lam,beta,delta,gama))
        int_val1 = int_p1[0]
        Cfiber1 = 2*mu1*(1/(lam**4))*int_val1
        
        int_p2 = quad(integrand2, gama, lam, args=(lam,beta,delta,gama))
        int_val2 = int_p2[0]
        Cfiber2 = -4*mu1*(1/(lam**4))*int_val2
        
        Cfiber = Cfiber1+Cfiber2
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    for m in range(3): 
                        VM = vonmises.pdf(theta, kappa)
                        Cplus[j,k,l,m] += Cfiber*VM*a0[j]*a0[k]*a0[l]*a0[m]
        
    ET = Ciso + Cvol + Cplus
    return ET

In [25]:
Id=np.eye(3)
mu0=0.41
mu1=225.93


beta=5.53
delta=0.44 
gama=0.97
K = 5
kappa=3

F = np.array([[1.2, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0] ])

ET1 = ET(F)
print(ET1)

[[[[16.10304766  2.19262862  0.        ]
   [ 2.19262862  2.12469818  0.        ]
   [ 0.          0.          1.23187313]]

  [[ 2.19262862 -2.10432772  0.        ]
   [ 0.89282505  0.41840547  0.        ]
   [ 0.          0.          0.        ]]

  [[ 0.          0.         -2.99715277]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]]


 [[[ 2.19262862  0.89282505  0.        ]
   [-2.10432772  0.41840547  0.        ]
   [ 0.          0.          0.        ]]

  [[ 2.12469818  0.41840547  0.        ]
   [ 0.41840547 -2.08321767  0.        ]
   [ 0.          0.          1.98690132]]

  [[ 0.          0.          0.        ]
   [ 0.          0.         -4.31589999]
   [ 0.          0.          0.        ]]]


 [[[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [-2.99715277  0.          0.        ]]

  [[ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   [ 0.         -4.31589999  0.        

In [32]:
## function to get inverse of a symmetric 3x3 matrix
# note, the input is C in voigt notation, i.e. not the
# full 3x3 C matrix, but a vector with the 6 independent
# components of C 
def inverseC(C):
    Det = C[0]*(C[1]*C[2]-C[5]**2) -C[3]*(C[3]*C[2]-C[5]*C[4])+C[4]*(C[3]*C[5]-C[4]*C[1])
    a11 = C[2] * C[1] - C[5]**2  
    a12 = C[3] * C[4] - C[2] * C[3]  
    a13 = C[3] * C[5] - C[4] * C[1]  
    a22 = C[2] * C[0] - C[4]**2  
    a23 = C[3] * C[4] - C[0] * C[5]
    a33 = C[0] * C[1] - C[3]**2
    return 1/Det*(np.array([a11,a22,a33,a12,a13,a23]))

## Function to calculate second Piola Kirchhoff Stress
# note, the input is C in voigt notation, i.e. not the
# full 3x3 C matrix, but a vector with the 6 independent
# components of C. Likewise, the output is a 6x1 vector 
def compute_S(C):
    Cmat = np.array([[C[0],C[3],C[4]],[C[3],C[1],C[5]],[C[4],C[5],C[2]]])
    cinv = inverseC(C)
    J2 = C[0]*(C[1]*C[2]-C[5]**2) -C[3]*(C[3]*C[2]-C[5]*C[4])+C[4]*(C[3]*C[5]-C[4]*C[1])
    J = sqrt(J2)
    I = np.array([1,1,1,0,0,0])
    I1 = C[0]+C[1]+C[2]
    Sp = np.zeros([3,3])
    # W Isochoric
    Siso=2*(J**(-2.0/3.0))*(mu0)*(I-(1./3.)*I1*(cinv))

    # W Volume
    p = 2*K*(J-1)
    Svol = p*J*cinv
    
    for i in range(10):
        theta=(i/10.0)*math.pi
        a0 = np.array([math.cos(theta),math.sin(theta),0])
        d_a0 = np.outer(a0,a0)
        lam = np.sqrt(np.dot(a0,np.dot(Cmat,a0)))
       
        
        int_p = quad(integrand, gama, lam, args=(lam,beta,delta,gama))
        int_val = int_p[0]
        Sfiber = 2*mu1*(1.0/(lam**2))*int_val
        
        VM = vonmises.pdf(theta, kappa)
    
        Sp += Sfiber*d_a0*VM
    Sfvoight = np.array([Sp[0,0], Sp[1,1], Sp[2,2], Sp[0,1], Sp[0,2], Sp[1,2]])
        
    S = Siso + Svol + Sfvoight
    
    return S

# Compute the elasticity tensor in voigt notation, which is a 6x6 matrix
# the input is C in voigt notation. Note, technically this is not the 
# elasticity tensor because there is a factor of 2 missing, we dont need
# the factor of 2 because we are solving for C instead of Green Lagrange
# strain, which is what appears in the weak form
def compute_CC(C):
    Cmat = np.array([[C[0],C[3],C[4]],[C[3],C[1],C[5]],[C[4],C[5],C[2]]])
    voigt_i_I = [0,1,2,0,0,1]
    voigt_j_I = [0,1,2,1,2,2]
    CCvoigt_iso = np.zeros((6,6))
    CCvoigt_vol = np.zeros((6,6))
    CCvoigt_fiber = np.zeros((6,6))
    cinv = inverseC(C)
    cinvmat = np.array([[cinv[0],cinv[3],cinv[4]],[cinv[3],cinv[1],cinv[5]],[cinv[4],cinv[5],cinv[2]]])
    J2 = C[0]*(C[1]*C[2]-C[5]**2) -C[3]*(C[3]*C[2]-C[5]*C[4])+C[4]*(C[3]*C[5]-C[4]*C[1])
    J = sqrt(J2)
    Jm = np.sqrt(np.linalg.det(Cmat))
    Id = np.array([[1.,0.,0.], [0.,1.,0.], [0.,0.,1]])
    I1 = C[0]+C[1]+C[2]
    p = 2*K*(Jm-1)
    
    for A in range(6):
        for B in range(6):
            a = voigt_i_I[A]
            b = voigt_j_I[A]
            c = voigt_i_I[B]
            d = voigt_j_I[B]

            #CCvoigt_iso[A,B] = 2*(Jm**(-2./3))*mu0*((-2./3)*Id[a,b]*cinvmat[c,d] + (2./9)*I1*cinvmat[a,b]*cinvmat[c,d] - (2./3)*cinvmat[a,b]*Id[c,d] + (2./3)*cinvmat[a,c]*cinvmat[b,d])
            CCvoigt_iso[A,B] = 2*(Jm**(-2./3))*mu0*((-1./3)*Id[a,b]*cinvmat[c,d] + (1./9)*I1*cinvmat[a,b]*cinvmat[c,d] - (1./3)*cinvmat[a,b]*Id[c,d] + (1./3)*I1*cinvmat[a,c]*cinvmat[b,d])

    for A in range(6):
        for B in range(6):
            e= voigt_i_I[A]
            f = voigt_j_I[A]
            g= voigt_i_I[B]
            h = voigt_j_I[B]
            
            # CCvoigt_vol[A,B] = 2*((1.0/2.0)*p*Jm*cinvmat[e,f]*cinvmat[g,h] - p*Jm*cinvmat[e,g]*cinvmat[f,h])
            CCvoigt_vol[A,B] = 2*((1.0/2.0)*(K*Jm*(2*Jm-1))*cinvmat[e,f]*cinvmat[g,h] - p*Jm*cinvmat[e,g]*cinvmat[f,h])
            
    for i in range(10):
        theta=(i/10.0)*math.pi
        a0 = np.array([math.cos(theta),math.sin(theta),0])
        d_a0 = np.outer(a0,a0)
        lam = np.sqrt(np.dot(a0,np.dot(Cmat,a0)))
       
        
        int_p1 = quad(integrand1, gama, lam, args=(lam,beta,delta,gama))
        int_val1 = int_p1[0]
        Cfiber1 = 2*mu1*(1.0/(lam**4))*int_val1
        
        int_p2 = quad(integrand2, gama, lam, args=(lam,beta,delta,gama))
        int_val2 = int_p2[0]
        Cfiber2 = -4.0*mu1*(1.0/(lam**4))*int_val2
        
        Cfiber = Cfiber1+Cfiber2
            
        for A in range(6):
            for B in range(6):
                j= voigt_i_I[A]
                k = voigt_j_I[A]
                l= voigt_i_I[B]
                m = voigt_j_I[B]
                VM = vonmises.pdf(theta, kappa)
                CCvoigt_fiber[A,B] += Cfiber*VM*a0[j]*a0[k]*a0[l]*a0[m] 
            
    ET = CCvoigt_iso + CCvoigt_vol+ CCvoigt_fiber
        
    return ET

In [33]:
Fmat = np.array([[1.01, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0] ])
Cmat = np.dot(Fmat,Fmat.transpose())
Cvoigt = np.array([Cmat[0,0], Cmat[1,1], Cmat[2,2], Cmat[0,1], Cmat[0,2], Cmat[1,2]])
print(Cvoigt)
#print(np.linalg.det(Cmat))
print('Stress')
Svoigt = compute_S(Cvoigt)
print(compute_S(Cvoigt))
print('Tangent')
CCvoigt = compute_CC(Cvoigt)
print(compute_CC(Cvoigt))

[1.0201 1.     1.     0.     0.     0.    ]
Stress
[1.09727981e-01 9.55448918e-02 9.55423241e-02 3.85237127e-06
 0.00000000e+00 0.00000000e+00]
Tangent
[[5.27689803e+00 4.78001931e+00 4.77976217e+00 4.82970073e-04
  0.00000000e+00 0.00000000e+00]
 [4.78001931e+00 5.49953364e+00 4.88129306e+00 1.75224517e-04
  0.00000000e+00 0.00000000e+00]
 [4.77976217e+00 4.88129306e+00 5.49932923e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.82970073e-04 1.75224517e-04 0.00000000e+00 6.06115549e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.05858409e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 6.18036163e-01]]


In [34]:
# Newton Raphson to solve given the stress tensor 
resnorm = 1
iter = 0
tol=1e-3

Svoigt1 = Svoigt
Csol = np.array([1.,1.,1.,0.,0.,0.])
while resnorm>tol and iter<30:
    CCvoigt = compute_CC(Csol)
    Ssol = compute_S(Csol)
    Res = Svoigt1 - Ssol
    Csol = Csol + np.linalg.solve(CCvoigt,Res)
    print('Csol')
    print(Csol)
    resnorm = np.linalg.norm(Res)
    iter+=1 
    print('iter %i'%iter)
    print('Residual norm: %f'%resnorm)
print('C_sol')
print(Csol)
print('C_real')
print(Cvoigt)

Csol
[1.01820550e+00 1.00092377e+00 1.00092391e+00 2.48912921e-07
 0.00000000e+00 0.00000000e+00]
iter 1
Residual norm: 0.174059
Csol
[ 1.02040253e+00  9.99849839e-01  9.99850086e-01 -7.51439158e-07
  0.00000000e+00  0.00000000e+00]
iter 2
Residual norm: 0.001616
Csol
[1.02005087e+00 1.00002408e+00 1.00002399e+00 2.56711332e-07
 0.00000000e+00 0.00000000e+00]
iter 3
Residual norm: 0.000262
C_sol
[1.02005087e+00 1.00002408e+00 1.00002399e+00 2.56711332e-07
 0.00000000e+00 0.00000000e+00]
C_real
[1.0201 1.     1.     0.     0.     0.    ]
